In [1]:
import numpy as np
import pandas as pd



In [68]:
# initialize fees
FEE_F = 0.003
FEE_G = 0.0025

# initialize basic arrays and dictionaries
q = {'ab': [0., 0.], 'ac': [0., 0.], 'bc': [0.,0.]}
swap_amt = [0.]*2


In [69]:

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)

init_state = {
    'exch': ['ab', 'ac', 'bc'],
    'tok0': [0.]*3,
    'tok1': [0.]*3,
    'res0': [q['ab'][0], q['ac'][0], q['bc'][0]],
    'res1': [q['ab'][1], q['ac'][1], q['bc'][1]],
    'reason': ["initialize"]*3
}

ledger_df = pd.DataFrame(init_state)

print(ledger_df)


  exch  tok0  tok1   res0   res1      reason
0   ab   0.0   0.0  100.0  100.0  initialize
1   ac   0.0   0.0  200.0  200.0  initialize
2   bc   0.0   0.0  250.0  250.0  initialize


In [70]:
def psi(x):
    assert x >= 0
    return 1/(1+x)

def phi(x):
    assert x >= 0
    assert x < 1
    return 1/(1-x)

def swap(data_):
    # data_ = { 'reserves': q['ab'], 'legs': [10., 0.], 'direction': 'forward', 'fee': FEE_F}
    f_ = data_['fee']
    d_ = data_['legs']
    r_ = data_['reserves']
    direction:str = data_['direction']

    assert r_[0] > 0
    assert r_[1] > 0

    # The in_leg has the non-zero entry
    if d_[0] > 0:
        assert d_[1] == 0
        in_leg = 0
        out_leg = 1
    else:
        assert d_[0] == 0
        in_leg = 1
        out_leg = 0

    q_in = r_[in_leg]
    q_out = r_[out_leg]

    if direction == 'backward':
        # backward swap:
        #     find the input that produces the output
        #     the output is known, so this is the "in_leg" which has a non-zero entry
        d_out = d_[in_leg]
        result_idx = out_leg
        d_in = float((1 / (1 - f_)) * (q_in / q_out) * phi(d_out / q_out) * d_out)
        d_[out_leg] = d_in
    elif direction == 'forward':
        # forward swap:
        #     find the output produced by the input
        #     the input is known, so this is the "in_leg" which has a non-zero entry
        d_in = d_[in_leg]
        result_idx = out_leg
        d_in = (1 - f_) * d_in
        d_out = float((q_out / q_in) * psi(d_in / q_in) * d_in)
        d_[out_leg] = d_out
    else:
        print('unknown direction')
        return

    r_[out_leg] -= d_out
    r_[in_leg]  += d_in
    return {'reserves': r_, 'legs': d_, 'result_idx': result_idx }

# a->b route
#
q = reset_reserves(q)

# Tests
if True:
    # swap a->b
    reason = 'route1: a->b'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [10., 0.],
                  'direction': 'forward',
                  'fee': FEE_F})
    print(reason)
    print(swap_out)
    route1_b_tokens = swap_out['legs'][swap_out['result_idx']]

    # swap c->b
    reason = 'cycle: c->b '
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [route1_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    cycle_c1_tokens = swap_out['legs'][swap_out['result_idx']]
    print(reason)
    print(swap_out)
    print(f"cycle_c1_tokens: {cycle_c1_tokens}")


route1: a->b
{'reserves': [109.97, 90.93389106119851], 'legs': [10.0, 9.066108938801491], 'result_idx': 1}
cycle: c->b 
{'reserves': [259.4355645298258, 240.9338910611985], 'legs': [9.066108938801491, 9.435564529825822], 'result_idx': 1}
cycle_c1_tokens: 9.435564529825822


In [71]:
# a->b route1
q = reset_reserves(q)

reason = 'route1: a->b'
exch = 'ab'
swap_out = swap({ 'reserves': q[exch],
                  'legs': [10., 0.],
                  'direction': 'forward',
                  'fee': FEE_F})
ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
route1_b_tokens = swap_out['legs'][swap_out['result_idx']]

In [72]:
print(ledger_df)

  exch  tok0      tok1    res0        res1        reason
0   ab   0.0  0.000000  100.00  100.000000    initialize
1   ac   0.0  0.000000  200.00  200.000000    initialize
2   bc   0.0  0.000000  250.00  250.000000    initialize
3   ab  10.0  9.066109  109.97   90.933891  route1: a->b


In [73]:
# a->c->b route2

q = reset_reserves(q)

# swap a->c
reason = 'route2: a->c'
exch = 'ac'
swap_out = swap({ 'reserves': q[exch],
                  'legs': [10., 0.],
                  'direction': 'forward',
                  'fee': FEE_F})
ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]

# record route2 c-token result
route2_c_tokens = swap_out['legs'][swap_out['result_idx']]
print(f"route2_c_tokens: {route2_c_tokens}")

# swap c->b
reason = 'route2: c->b'
exch = 'bc'
swap_out = swap({ 'reserves': q[exch],
                  'legs': [0., route2_c_tokens], # note: incoming leg is c-tokens, which is index 1
                  'direction': 'forward',
                  'fee': FEE_F})
ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]

route2_b_tokens = swap_out['legs'][swap_out['result_idx']]
print(f"route2_b_tokens: {route2_b_tokens}")

print(f"Difference route1 minus route2: {route1_b_tokens - route2_b_tokens}")

route2_c_tokens: 9.496594751631186
route2_b_tokens: 9.122609663880215
Difference route1 minus route2: -0.056500725078723946


In [74]:
print(ledger_df)

  exch      tok0      tok1       res0        res1        reason
0   ab   0.00000  0.000000  100.00000  100.000000    initialize
1   ac   0.00000  0.000000  200.00000  200.000000    initialize
2   bc   0.00000  0.000000  250.00000  250.000000    initialize
3   ab  10.00000  9.066109  109.97000   90.933891  route1: a->b
4   ac  10.00000  9.496595  209.97000  190.503405  route2: a->c
5   bc   9.12261  9.496595  240.87739  259.468105  route2: c->b


In [75]:
# cycle

if route2_b_tokens - route1_b_tokens > 0:
    cycle = 'c->b->a->c'
else:
    cycle = 'c->a->b->c'

print(f"cycle: {cycle}")

q = reset_reserves(q)

if cycle == 'c->b->a->c':
    # swap c->b
    reason = 'cycle: c->b should be route1-b'
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [route1_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
    cycle_c1_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"cycle_c1_tokens: {cycle_c1_tokens}")

    # swap b->a
    reason = 'cycle: b->a should be route1-a'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0., route1_b_tokens],
                      'direction': 'backward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
    cycle_a_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"cycle_a_tokens: {cycle_a_tokens}")

    # swap a->c
    reason = 'cycle: a->c'
    exch = 'ac'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [cycle_a_tokens, 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
    cycle_c2_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"cycle_c2_tokens: {cycle_c2_tokens}")

    print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")

cycle: c->b->a->c
cycle_c1_tokens: 9.435564529825822
cycle_a_tokens: 9.999999999999998
cycle_c2_tokens: 9.496594751631184
Difference c2 minus c1: 0.06103022180536222
